In [1]:
import pandas as pd

In [2]:
%pip install openpyxl

Looking in indexes: https://pypi.doubanio.com/simple

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
s=pd.read_excel("new_character_data_.xlsx")
s

,UID,Name,Link (only from pdb),CID,Source,Description (only from pdb),Event Childhood,Event Teenage,Event Pre-Hero,Event Hero (1),...,Dilemma(3) Reveal Identity or Protect Loved Ones,Dilemma(4) Join or Fight Against the Government,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,True choice
0,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/1...,148,MCU: The Heroes,Tony Stark was a character portrayed by #Rober...,your father Howard Seeing that you could achie...,"when you was 21, your parents are killed in a ...",when after break free from imprisoned by a ter...,Now that you was publicly revealed yourself as...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A A A B C A
1,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/1...,1982,Marvel Hero,"Iron Man (Anthony Edward ""Tony"" Stark) is a fi...",which pivotal decision did you that significan...,what pivotal decision did you make that influe...,Before fully committing to becoming a superher...,"During the events of the ""Civil War,"" you made...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B C C A C D
2,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/3...,376301,Marvel Hero,"Anthony Edward \""Tony\"" Stark was a billionair...",your father Howard Seeing that you could achie...,"when you was 21, your parents are killed in a ...","In Afghanistan, you was ambushed by the Ten Ri...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/2...,2589,MCU: The Heroes,After getting bitten by a scientifically-alter...,During your childhood visit to the Stark Expo ...,how do you balances your double life as Spide...,"Before you gained your spider powers, you had ...","During a mid-air battle, Spider-Man was caught...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\nC C A B B A\n
4,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/9...,920,The Amazing Spider-Man Duology,Peter Benjamin Parker is the superhero vigilan...,"When your parents, Richard and Mary Parker is ...","As Peter Parker, before gaining your spider po...",You've recently been gifted with amazing spide...,"you've discovered that your mentor, Dr. Curt C...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A B B D B C
5,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/1...,15917,Spider-Man (Franchise),This profile is strictly based on Peter Parker...,"As a young Peter Parker, before you became Spi...",You were a high school student at Midtown High...,you've just obtained critical evidence from a ...,While investigating the mysterious masked gunm...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B D B A A B
6,3,Captain America (Steve Rogers),https://www.personality-database.com/profile/1...,155,MCU: The Heroes,Steve Rogers (Captain America) is a fictional ...,"Walking down Tenth Avenue, you encounter a gro...",Inside a crowded cinema during a war documenta...,"In the midst of basic training, a grenade sudd...",The world's governments have presented the Sok...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D C D A B A
7,3,Captain America (Steve Rogers),https://www.personality-database.com/profile/1...,1983,Marvel Comics,"Captain America (Steven \""Steve\"" Rogers) is a...",When you were a young boy growing up in Brookl...,"As a teenager, before becoming Captain America...",During your time as a teenage soldier before b...,"You're on a mission during World War II, and y...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C A C B A B
8,4,Thor Odinson,https://www.personality-database.com/profile/1...,154,MCU: The Heroes,Thor Odinson is a character portrayed by Chris...,"""As a young prince of Asgard, you were raised ...","""During your teenage years, you and Loki frequ...","""On the day you were set to be crowned King of...","""Stripped of your powers and banished to Earth...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,Thor Odinson,https://www.personality-database.com/p

In [4]:
s.to_csv("filter_character_data.csv",index=False)

In [6]:
df=pd.read_csv("filter_character_data.csv")
df

,UID,Name,Link (only from pdb),CID,Source,Description (only from pdb),Event Childhood,Event Teenage,Event Pre-Hero,Event Hero (1),...,Dilemma(3) Reveal Identity or Protect Loved Ones,Dilemma(4) Join or Fight Against the Government,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,True choice
0,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/1...,148,MCU: The Heroes,Tony Stark was a character portrayed by #Rober...,your father Howard Seeing that you could achie...,"when you was 21, your parents are killed in a ...",when after break free from imprisoned by a ter...,Now that you was publicly revealed yourself as...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A A A B C A
1,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/1...,1982,Marvel Hero,"Iron Man (Anthony Edward ""Tony"" Stark) is a fi...",which pivotal decision did you that significan...,what pivotal decision did you make that influe...,Before fully committing to becoming a superher...,"During the events of the ""Civil War,"" you made...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B C C A C D
2,1,Iron Man (Tony Stark),https://www.personality-database.com/profile/3...,376301,Marvel Hero,"Anthony Edward \""Tony\"" Stark was a billionair...",your father Howard Seeing that you could achie...,"when you was 21, your parents are killed in a ...","In Afghanistan, you was ambushed by the Ten Ri...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/2...,2589,MCU: The Heroes,After getting bitten by a scientifically-alter...,During your childhood visit to the Stark Expo ...,how do you balances your double life as Spide...,"Before you gained your spider powers, you had ...","During a mid-air battle, Spider-Man was caught...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\nC C A B B A\n
4,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/9...,920,The Amazing Spider-Man Duology,Peter Benjamin Parker is the superhero vigilan...,"When your parents, Richard and Mary Parker is ...","As Peter Parker, before gaining your spider po...",You've recently been gifted with amazing spide...,"you've discovered that your mentor, Dr. Curt C...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A B B D B C
5,2,Spider-Man (Peter Parker),https://www.personality-database.com/profile/1...,15917,Spider-Man (Franchise),This profile is strictly based on Peter Parker...,"As a young Peter Parker, before you became Spi...",You were a high school student at Midtown High...,you've just obtained critical evidence from a ...,While investigating the mysterious masked gunm...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B D B A A B
6,3,Captain America (Steve Rogers),https://www.personality-database.com/profile/1...,155,MCU: The Heroes,Steve Rogers (Captain America) is a fictional ...,"Walking down Tenth Avenue, you encounter a gro...",Inside a crowded cinema during a war documenta...,"In the midst of basic training, a grenade sudd...",The world's governments have presented the Sok...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D C D A B A
7,3,Captain America (Steve Rogers),https://www.personality-database.com/profile/1...,1983,Marvel Comics,"Captain America (Steven \""Steve\"" Rogers) is a...",When you were a young boy growing up in Brookl...,"As a teenager, before becoming Captain America...",During your time as a teenage soldier before b...,"You're on a mission during World War II, and y...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C A C B A B
8,4,Thor Odinson,https://www.personality-database.com/profile/1...,154,MCU: The Heroes,Thor Odinson is a character portrayed by Chris...,"""As a young prince of Asgard, you were raised ...","""During your teenage years, you and Loki frequ...","""On the day you were set to be crowned King of...","""Stripped of your powers and banished to Earth...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,Thor Odinson,https://www.personality-database.com/p

In [7]:
df.columns

Index(['UID', 'Name', 'Link (only from pdb)', 'CID', 'Source',
       'Description (only from pdb)', 'Event Childhood', 'Event Teenage',
       'Event Pre-Hero', 'Event Hero (1)', 'Event Hero (2)', 'Event Hero (3)',
       'Dilemma(1) Save One or Save Many', 'Dilemma (2) Hero or Villain?',
       'Dilemma(3) Reveal Identity or Protect Loved Ones',
       'Dilemma(4) Join or Fight Against the Government', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'True choice'],
      dtype='object')